## シンプルなLLMアプリケーション

### 必要なライブラリのインストール

In [1]:
%pip install -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.


### CohereのSDKを利用したLLMアプリケーション

`load_dotenv`で.envに設定したCOHERE_API_KEYを読み込みます。  
まずはLangChainを利用せず、Cohere社が提供するSDKを利用して、LLMを利用してみます。


In [1]:

import cohere
from dotenv import load_dotenv

load_dotenv("../.env")

co = cohere.Client()

chat = co.chat(
    message="おいらは誰どす？",
    model="command-r"
)

chat

/data/ghq/github.com/codex-odyssey/llm-observability/venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


NonStreamedChatResponse(text='申し訳ありませんが、私はあなたが誰なのか分かりません。あなたは自分が誰だと思いますか？ \n\nもしご自分が何者なのか、お名前やご身分をお知りになりたい場合、私に教えていただければ幸いです。お答えできる範囲でお手伝いいたします。', generation_id='81e63b63-2789-4383-95d3-503064f46414', citations=None, documents=None, is_search_required=None, search_queries=None, search_results=None, finish_reason='COMPLETE', tool_calls=None, chat_history=[Message_User(message='おいらは誰ですか？', tool_calls=None, role='USER'), Message_Chatbot(message='申し訳ありませんが、私はあなたが誰なのか分かりません。あなたは自分が誰だと思いますか？ \n\nもしご自分が何者なのか、お名前やご身分をお知りになりたい場合、私に教えていただければ幸いです。お答えできる範囲でお手伝いいたします。', tool_calls=None, role='CHATBOT')], prompt=None, meta=ApiMeta(api_version=ApiMetaApiVersion(version='1', is_deprecated=None, is_experimental=None), billed_units=ApiMetaBilledUnits(input_tokens=7.0, output_tokens=69.0, search_units=None, classifications=None), tokens=ApiMetaTokens(input_tokens=73.0, output_tokens=69.0), warnings=None), response_id='36701525-fec9-4703-a671-e687f1906b48')

chat_historyを利用した例

In [3]:
chat = co.chat(
    chat_history=[
        {"role": "User", "message": "おっす!おらはBigBaByだぞ!BBと呼んじくんな"},
        {"role": "Chatbot", "message": "分かりました。あなたのことをBBと呼びます"},
    ],
    message="おいらは誰どす？",
    model="command-r"
)

chat

NonStreamedChatResponse(text='あなたはBigBaByです。略してBBです。', generation_id='6570e1a4-1121-4990-bdfb-51d4eb200ebf', citations=None, documents=None, is_search_required=None, search_queries=None, search_results=None, finish_reason='COMPLETE', tool_calls=None, chat_history=[Message_User(message='おっす!おらはBigBaByだぞ!BBと呼んじくんな', tool_calls=None, role='USER'), Message_Chatbot(message='分かりました。あなたのことをBBと呼びます', tool_calls=None, role='CHATBOT'), Message_User(message='おいらは誰どす？', tool_calls=None, role='USER'), Message_Chatbot(message='あなたはBigBaByです。略してBBです。', tool_calls=None, role='CHATBOT')], prompt=None, meta=ApiMeta(api_version=ApiMetaApiVersion(version='1', is_deprecated=None, is_experimental=None), billed_units=ApiMetaBilledUnits(input_tokens=35.0, output_tokens=12.0, search_units=None, classifications=None), tokens=ApiMetaTokens(input_tokens=107.0, output_tokens=12.0), warnings=None), response_id='c39305a7-4c7f-4133-a59e-daf71f523568')

### LangChainを利用した例

### ChatModel

In [5]:
from langchain_cohere import ChatCohere

model = ChatCohere(model="command-r")


### Message

In [6]:

from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    HumanMessage("おっす!おらはBigBaByだぞ!BBと呼んじくんな"),
    SystemMessage("分かりました。あなたのことをBBと呼びます"),
    HumanMessage("おいらは誰どす？"),
]

model.invoke(messages)

AIMessage(content='あなたはBigBaByさんです。略してBBさんですね。', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'f67b720a-b5b8-4986-b763-7313df95d016', 'token_count': {'input_tokens': 107.0, 'output_tokens': 17.0}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'f67b720a-b5b8-4986-b763-7313df95d016', 'token_count': {'input_tokens': 107.0, 'output_tokens': 17.0}}, id='run-187ab868-2b79-4cd3-8a86-03ef0a7f7dc5-0', usage_metadata={'input_tokens': 107, 'output_tokens': 17, 'total_tokens': 124})

### Prompt Template

In [8]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Tell me a joke about {topic}")

prompt_template.invoke({"topic": "cats"})

StringPromptValue(text='Tell me a joke about cats')

In [9]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "Tell me a joke about {topic}")
])

prompt = prompt_template.invoke({"topic": "cats"})
prompt

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant'), HumanMessage(content='Tell me a joke about cats')])

In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder("msgs")
])

prompt = prompt_template.invoke({"msgs": [HumanMessage(content="hi!")]})
prompt

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant'), HumanMessage(content='hi!')])

In [13]:
model.invoke(prompt)

AIMessage(content="Hi! How's it going? I'm here to help you in any way I can. Do you have a question or a task for me?", additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '375f91e8-4b0f-4312-83d6-62eecb9a5d40', 'token_count': {'input_tokens': 76.0, 'output_tokens': 30.0}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '375f91e8-4b0f-4312-83d6-62eecb9a5d40', 'token_count': {'input_tokens': 76.0, 'output_tokens': 30.0}}, id='run-dc2da997-c21f-4a63-9053-18f37cd0faad-0', usage_metadata={'input_tokens': 76, 'output_tokens': 30, 'total_tokens': 106})

### OutputParser

In [14]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field

# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")


# And a query intented to prompt a language model to populate the data structure.
joke_query = "ジョークを言ってください"

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})

{'setup': 'Why did the chicken cross the road?',
 'punchline': "I'm not sure, but the road will have its vengeance!"}

In [ ]:
parser.get_format_instructions()